# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "bay-wheels").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [5]:
import pandas as pd

stations_df = pd.read_csv("bike_stations.csv")
bay_wheels = stations_df[stations_df["network"] == "bay-wheels"]
sf_count = bay_wheels["city"].value_counts()[0]
print(f"SF count of bikes {sf_count}")

SF count of bikes 508


## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [6]:
import requests

request = {
    "origins": [],
    "destinations": [{
    "latitude": 37.8024,
    "longitude": -122.4058
    }],
    "travelMode": "Driving"
}
for location in bay_wheels.iterrows():
    request["origins"].append({
        "latitude": location[1]["latitude"],
        "longitude": location[1]["longitude"]
        })

response = requests.post("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AoS8mFZV01fqp6GEjzmlg-Dy0M_OTu2EZcZXQJgHYL-mPDgJnWPO9CVVBqVTa8wK", json=request)

In [7]:
response_df = pd.json_normalize(response.json()["resourceSets"][0]["resources"][0]["results"])
min_distance = response_df["travelDistance"].idxmin()
print(bay_wheels.iloc[min_distance]["name"])

Lombard St at Columbus Ave


## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.